In [1]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint

def main(communicator):
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)

Note that we're using a very small field (`order=127`) and a very low-precision encoding (`precision=2`) to make the values in the logged output easier to read.

In [2]:
import logging

from cicada import transcript

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.netmsg_handler())
    SocketCommunicator.run(fn=main, world_size=3);

Player 2: --> 0 PRZS 5838487747918109027
Player 0: --> 1 PRZS 5189197214688001201
Player 1: --> 2 PRZS 8951291160511430181
Player 2: <-- 1 PRZS 8951291160511430181
Player 0: <-- 2 PRZS 5838487747918109027
Player 1: <-- 0 PRZS 5189197214688001201
Player 2: --> 0 GATHER 18
Player 0: --> 0 GATHER 103
Player 1: --> 0 GATHER 26
Player 2: --> 1 GATHER 18
Player 2: --> 2 GATHER 18
Player 0: <-- 2 GATHER 18
Player 1: --> 1 GATHER 26
Player 0: <-- 0 GATHER 103
Player 2: <-- 2 GATHER 18
Player 1: <-- 2 GATHER 18
Player 0: <-- 1 GATHER 26
Player 1: <-- 1 GATHER 26
Player 0: --> 1 GATHER 103
Player 0: --> 2 GATHER 103
Player 1: <-- 0 GATHER 103
Player 2: <-- 0 GATHER 103
Player 1: --> 2 GATHER 26
Player 2: <-- 1 GATHER 26


In [3]:
def main(communicator):
    transcript.log("Let's setup additive sharing!")
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.netmsg_handler())
    SocketCommunicator.run(fn=main, world_size=3);

Player 0: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
Player 0: --> 1 PRZS 7689222575894296698
Player 2: --> 0 PRZS 6672346396073719137
Player 1: --> 2 PRZS 3583060991058592230
Player 2: <-- 1 PRZS 3583060991058592230
Player 1: <-- 0 PRZS 7689222575894296698
Player 0: <-- 2 PRZS 6672346396073719137
Player 2: Let's share some secrets!
Player 0: Let's share some secrets!
Player 1: Let's share some secrets!
Player 2: Let's add some secrets!
Player 0: Let's add some secrets!
Player 1: Let's add some secrets!
Player 2: Let's reveal the results!
Player 0: Let's reveal the results!
Player 1: Let's reveal the results!
Player 2: --> 0 GATHER 76
Player 0: --> 0 GATHER 0
Player 1: --> 0 GATHER 71
Player 2: --> 1 GATHER 76
Player 0: <-- 2 GATHER 76
Player 1: --> 1 GATHER 71
Player 2: --> 2 GATHER 76
Player 0: <-- 0 GATHER 0
Player 1: <-- 2 GATHER 76
Player 2: <-- 2 GATHER 76
Player 0: <-- 1 GATHER 71
Player 1: <-- 1 GATHER 71
Player

In [4]:
with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.netmsg_handler(fmt="# {processName}: {msg}", netmsgfmt="{processName},{netmsg.verb},{netmsg.src},{netmsg.dst},{netmsg.tag},{netmsg.payload}"))
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
Player 2,sent,2,0,PRZS,3935484400371542738
Player 1,sent,1,2,PRZS,8562152921621681264
Player 0,sent,0,1,PRZS,1095542009893839754
Player 0,received,2,0,PRZS,3935484400371542738
Player 1,received,0,1,PRZS,1095542009893839754
Player 2,received,1,2,PRZS,8562152921621681264
# Player 0: Let's share some secrets!
# Player 1: Let's share some secrets!
# Player 2: Let's share some secrets!
# Player 2: Let's add some secrets!
# Player 0: Let's add some secrets!
# Player 1: Let's add some secrets!
# Player 2: Let's reveal the results!
# Player 0: Let's reveal the results!
# Player 1: Let's reveal the results!
Player 2,sent,2,0,GATHER,86
Player 0,sent,0,0,GATHER,54
Player 1,sent,1,0,GATHER,7
Player 2,sent,2,1,GATHER,86
Player 0,received,2,0,GATHER,86
Player 1,sent,1,1,GATHER,7
Player 2,sent,2,2,GATHER,86
Player 0,received,0,0,GATHER,54
Player 1,received,2,1,GATHER,86
Player

If you look carefully, you can see that this produces two nearly identical events for each message (once when the message is sent, and once when the message is received).  If you wish to eliminate the duplication, e.g. by only logging messages when they're sent, you can specify that too:

In [5]:
with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.netmsg_handler(received=False, fmt="# {processName}: {msg}", netmsgfmt="{processName},{netmsg.verb},{netmsg.src},{netmsg.dst},{netmsg.tag},{netmsg.payload}"))
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
Player 0,sent,0,1,PRZS,9110473576963038339
Player 1,sent,1,2,PRZS,770007444957641767
Player 2,sent,2,0,PRZS,6946501331644846854
# Player 1: Let's share some secrets!
# Player 0: Let's share some secrets!
# Player 2: Let's share some secrets!
# Player 1: Let's add some secrets!
# Player 2: Let's add some secrets!
# Player 0: Let's add some secrets!
# Player 1: Let's reveal the results!
# Player 2: Let's reveal the results!
# Player 0: Let's reveal the results!
Player 1,sent,1,0,GATHER,91
Player 2,sent,2,0,GATHER,9
Player 0,sent,0,0,GATHER,47
Player 1,sent,1,1,GATHER,91
Player 2,sent,2,1,GATHER,9
Player 0,sent,0,1,GATHER,47
Player 2,sent,2,2,GATHER,9
Player 0,sent,0,2,GATHER,47
Player 1,sent,1,2,GATHER,91


In [6]:
from cicada.arithmetic import Field

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.code_handler())
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype=object))
bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 55717778750175161712753607949265336342, 'inc': 65620565693107867681799285943225391795}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([158, 235, 183], dtype=object))
lhs = numpy.array([1, 1, 1], dtype=object)
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([158, 235, 183], dtype=object))
cicada.transcript.assert_equal(lhs, numpy.array([32, 109, 57], dtype=object))
